In [2]:
import os
os.chdir("/Users/candicechan/Desktop/sta141b final proj")
print(os.getcwd())

/Users/candicechan/Desktop/sta141b final proj


In [4]:
import requests
import pandas as pd
import time

url = "https://api.yelp.com/v3/businesses/search"
api_key = "MsdajOA6rF0Wt3lBAKm8dqjSBy-X2aIOlKT8BFWkhV_-fuYTMWHHNKFfDRb-Ff3WCXvBb7VQyK9RtVvn_mXnp1HEcaVNhB9Kh_yPHFPuJBnIPoDPi_2NBOjkVbfPZ3Yx"
headers = {
    'Authorization': 'Bearer ' + api_key
}

In [ ]:
# EXTRACTING RESTAURANT INFO USING YELP API
all_restaurants = []
max_results = 240  # Yelp's hard limit
batch_size = 50  # Max results per request

# Loop with adjusted offset to avoid exceeding 240
for offset in range(0, 200, batch_size):
    params = {
        "location": "Davis, CA",
        "term": "restaurants",
        "limit": batch_size,
        "offset": offset
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        businesses = data.get("businesses", [])

        if not businesses:
            break

        for biz in businesses:
            city = biz["location"].get("city", "").strip().lower()

            # Keep only restaurants in Davis
            if city == "davis":
                price = biz.get("price", "Unknown")

                all_restaurants.append({
                    "name": biz["name"],
                    "rating": biz["rating"],
                    "review_count": biz["review_count"],
                    "address": ", ".join(biz["location"]["display_address"]),
                    "cuisine": [cat["title"] for cat in biz["categories"]],
                    "price": price,
                    "yelp_url": f"https://www.yelp.com/biz/{biz['id']}"
                })

        print(f"Retrieved {len(all_restaurants)} restaurants so far...")

        time.sleep(1)  # Avoid hitting API rate limits
    else:
        print(f"Error: {response.status_code}, {response.text}")
        break

df = pd.DataFrame(all_restaurants)

print(f"Total restaurants collected: {len(df)} (Only in Davis, CA)")

In [ ]:
# Remove the restaurants with "Unknown" price
# Load the fixed dataset
file_path = "yelp_restaurants_davis_fixed.csv"
df = pd.read_csv(file_path)

# Remove restaurants where price is "Unknown"
df_cleaned = df[df["price"] != "Unknown"]

# Save the cleaned dataset
output_path = "no_unknowns_yelp_restaurants_davis.csv"
df_cleaned.to_csv(output_path, index=False)

# Print confirmation
print(f"Original restaurant count: {len(df)}")
print(f"Cleaned restaurant count (price known): {len(df_cleaned)}")
print(f"Saved cleaned dataset to: {output_path}")